# Order book program

In [5]:
import queue as Q
import os

#
# COMMENT: You will probably want to change the folder
#
os.chdir("/Users/guoqi/Desktop/688 hw")

ncustomers=100     # number of customers assumed
trades_processed=0 # track number of trades processed
 
#
# Containers for bids and asks 
#
bidvector=[] # includes bid id's for entire bid history 
askvector=[] # includes ask id's for entire ask history

#
# Dictionary to look up a bid from a bidid
#
bid_dictionary={}
ask_dictionary={}

#
# Dictionary of customers
#
customer_dictionary={}

#
# The bid and ask priority queues.
#
bid_queue=Q.PriorityQueue()
ask_queue=Q.PriorityQueue()


#
# Functions used to format a time of day object
# I use these to ensure that numbers appear with
# the same number of digits - e.g.
#
# mystr2:
#     2 becomes 02
#    14 becomes 14
# mystr3:
#     0 becomes 000
#     8 becomes 008
#    17 becomes 017
#   123 becomes 123
#
def mystr2(n):
    if n<10:
        return("0"+str(n))
    return(str(n))
def mystr3(n):
    if n<10:
        return("00"+str(n))
    if n<100:
        return("0"+str(n))
    return(str(n))    
#
# time of day object
#
class timeofday:
    def __init__(self,hr,minute,sec,ms):
        self.hr=hr
        self.min=minute
        self.sec=sec
        self.ms=ms
    def __str__(self):
        tstr=mystr2(self.hr)+":"
        tstr+=mystr2(self.min)+":"
        tstr+=mystr2(self.sec)+":"
        tstr+=mystr3(self.ms)
        return(tstr)   
#####################################################################
#####################################################################
#####################################################################
#
# Problem 1: write a function called "timeless" to compare two time 
# time of day objects. 
#
# When called using timeless(time1,time2)
#
# This should return True if time1 is prior to time2
# and in case of ties, return False
#
# This does not need to be a class method. You can write a function 
# called timeless than takes two timeofday arguments and returns True/False
#
#####################################################################
#####################################################################

def timeless(time1,time2):
    T1=[int(x)for x in str(time1).split(":")]
    T2=[int(y)for y in str(time2).split(":")]
    for i in range(4):
        if T1[1]>T2[i]:
            return False
        elif T1[1]==T2[i]:
            continue
        else:
            return True
    return False
            



#
# class for storing a request: 
#
class request:
    def __init__(self,time,customerid,activate_or_deactivate,bid_or_ask,orderid,price,quantity):
        self.time=time
        self.customerid=customerid
        self.activate_or_deactivate=activate_or_deactivate
        self.bid_or_ask=bid_or_ask
        self.orderid=orderid
        self.price=price
        self.quantity=quantity
    def __str__(self):
        tstr=self.time.__str__()+" "
        tstr+=str(self.customerid)+" "
        tstr+=self.activate_or_deactivate+" "
        tstr+=self.bid_or_ask+" "
        tstr+=self.orderid+" "
        tstr+=str(self.price)+" "
        tstr+=str(self.quantity)
        return(tstr)

#
# class for bid objects - 
#
#    once a bid is created, its orderid and customerid, time and price
#      should never change
#
#    the quantity can change as a result of a trade being carried out
#
#    active will be set to true when the activate bid request arrives
#      but can be set to False if either: 
#
#          i) request to deactivate bid is made
#          ii) trade for the bid is fully completed (quantity becomes 0)
#
class bid:
    def __init__(self,orderid,customerid,time,price,quantity,active):
        self.orderid=orderid
        self.customerid=customerid 
        self.time=time
        self.price=price
        self.quantity=quantity
        self.active=active
    #
    # function for priority comparison of bids
    # note: lower value in the ordering => higher priority
    #       so if bidA has higher price than bidB we want
    #             __lt__(bidA,bidB) 
    #       to evaluate to True.
    #
    #######################################################
    #######################################################
    #
    # Problem 2: Write a less than function __lt__
    #    in the bid class for comparing one bid (self)  
    #    to another (other) so that
    #    
    #    __lt__(self,other) equals True if self gets higher 
    #    priority and false otherwise
    #   
    #######################################################
    def __lt__(self,other):
        if self.price==other.price:
            if timeless(self.time,other.time)==True:
                return True
            else:
                return False
        elif self.price>other.price:
            return True
        else:
            return False
    #######################################################
    #
    # make string for bid object
    #
    def __str__(self):
        tstr="bid:"
        tstr+=self.time.__str__()+" "
        tstr+=str(self.customerid)+" "
        if self.active:
            tstr+=" active "
        else:
            tstr+=" inactive "
        tstr+=self.orderid+" "
        tstr+=str(self.price)+" "
        tstr+=str(self.quantity)
        tstr+="\n"
        return(tstr)
#
# class for ask objects - 
#
#    once an ask is created, its orderid and customerid, time and price
#      should never change
#
#    the quantity can change as a result of a trade being carried out
#
#    active will be set to true when the activate ask request arrives
#      but can be set to False if either: 
#
#          i) request to deactivate bid is made
#          ii) trade for the bid is fully completed (quantity becomes 0)
#

class ask:
    def __init__(self,orderid,customerid,time,price,quantity,active):
        self.orderid=orderid
        self.customerid=customerid 
        self.time=time
        self.price=price
        self.quantity=quantity
        self.active=active
    #
    # function for priority comparison of asks
    # higher price implies lower priority
    #
    #######################################################
    #######################################################
    #
    #
    # Problem 3: Write a less than function __lt__
    #    in the ask class for comparing one ask (self)  
    #    to another (other) so that
    #    
    #    __lt__(self,other) equals True if self gets higher 
    #    priority and false otherwise
    #
    # make string for ask object
    #
    def __lt__(self,other):
        if self.price<other.price:
            return True
        elif self.price==other.price:
            if timeless(self.time,other.time):
                return True
            else:
                return False
        else:
            return False
    def __str__(self):
        tstr="ask:"
        tstr+=self.time.__str__()+" "
        tstr+=str(self.customerid)+" "
        if self.active:
            tstr+=" active "
        else:
            tstr+=" inactive "
        tstr+=self.orderid+" "
        tstr+=str(self.price)+" "
        tstr+=str(self.quantity)
        tstr+="\n"
        return(tstr)        
        
#
# Customer object
#
class customer:
    def __init__(self,customerid,nshares,cash,active_bidids,active_askids):
        self.customerid=customerid
        self.nshares=nshares
        self.cash=cash
        self.active_bidids=active_bidids
        self.active_askids=active_askids
    def __str__(self):
        tstr=self.customerid+" "
        tstr+=str(self.nshares)+" "
        tstr+=str(self.cash)
        tstr+="\n"
        return(tstr)
#
# update the queue
#
def update_queue():
    global bid_queue,ask_queue,customer_dictionary,trades_processed
    
    while True:
        
        #########################################################
        #
        # Problem 4
        #
        # Check whether the bid queue is empty and if it is, return
        #  from the update_queue() function     
        # Check whether the ask queue is empty and if it is, return
        #   from the update_queue() function      
        if bid_queue.empty()==True:
            return
        if ask_queue.empty()==True:
            return
         # 
        # Problem 5
        #
        # Set variable "bid" equal to the top priority bid from the bid queue without 
        # removing it from the queue 
        bid=bid_queue.queue[0]
        #
        # Problem 6
        #
        # If bid is inactive, pop it (remove it) from the queue and continue without
        # doing anything else in this while loop.
        if bid.active==False:
            bid_queue.get()
            continue
        #
        # Problem 7
        #
        # Set variable "ask" equal to the top priority ask from the ask queue without 
        # removing it from the queue
        ask=ask_queue.queue[0]
        #
        # Problem 8
        #
        # If ask is inactive, pop it (remove it) from the queue and continue without
        # doing anything else in this while loop.
        #
        if ask.active==False:
            ask_queue.get()
            continue
        #
        # Now we process the top priority bid,ask pairs just obtained
        # note that we don't modify the queue unless a trade occurs
        #
        # Problem 9
        #
        # check whether these bid and ask pairs lead to a trade and
        # return from the update_queue function if not
        #
        if bid.price<ask.price:
            return
        ###########################################################################
        ###########################################################################
        #
        # Here is where you might want to print the order book out before trade
        # is processed
        #
        ###########################################################################
        ###########################################################################
        #
        # A trade should take place so we process the trade
        #
        # update the trades_processed count 
        #
        trades_processed+=1
       
        #
        # Problem 10
        #
        # determine the price the bidder and asker should trade at
        # and call this variable "price"
        if timeless(bid.time,ask.time):
            price=bid.price
        else:
            price=ask.price
        #
        # Problem 11 
        # 
        # determine the quantity traded, i.e. the minimum of the bid and ask
        # quantities ask.quantity
        quantity=min(bid.quantity,ask.quantity)
        
        #
        # Problem 12 
        #
        # compute cash exchanged and store it in variable called cash_exchanged
        #
        cash_exchanged=quantity*price
        #
        # Problem 13
        # 
        # get the customer ids associated with the bids and asks
        # and stored them in variables called bidder_id and asker_id
        #
        bidder_id=bid.customerid
        asker_id=ask.customerid
    
        # 
        # Problem 14
        #
        # update customer's shares and cash resulting from the trade
        # warning - be careful to consider the possibility that
        # a trader is both the bidder and the asker
        customer_dictionary[bidder_id].nshares+=quantity
        customer_dictionary[bidder_id].nshares-=quantity
        customer_dictionary[bidder_id].cash-=cash_exchanged
        customer_dictionary[bidder_id].cash+=cash_exchanged
        
        #
        # Problem 15
        #
        # modify the bid and asks to reflect changes in quantities
        bid.quantity=bid.quantity-quantity
        ask.quantity=ask.quantity-quantity
        #
        # Problem 16
        #
        # if bid quantity becomes zero, we need to
        #    - deactivate it
        #    - remove it from the customer's active bid list
        #    - remove the bid from the bid queue
        if bid.quantity==0:
            bid.active=False
            customer_dictionary[bidder_id].active_bidids.remove(bid.orderid)
            bid_queue.get()
        #
        # Problem 17
        #
        # if ask quantity becomes zero, we need to
        #    - deactivate it
        #    - remove it from the customer's active ask list
        #    - remove the ask from the bid queue
        #
        if ask.quantity==0:
            ask.active=False
            customer_dictionary[asker_id].active_askids.remove(ask.orderid)
            ask_queue.get()
            

#
# instantiate a new bid
#
def activate_new_bid(req):
    global bid_queue, customer_dictionary
    # instantiate a new bid
    newbid=bid(req.orderid,req.customerid,req.time,req.price,req.quantity,True)
    # add to the bid dictionary
    bid_dictionary[req.orderid]=newbid
    # put it in the queue
    bid_queue.put(newbid)
    # update the customer's active bid list
    customer_dictionary[req.customerid].active_bidids.append(req.orderid)
#
# instantiate a new ask and make it active
#      
def activate_new_ask(req):
    global ask_queue, customer_dictionary
    # instantiate a new ask
    newask=ask(req.orderid,req.customerid,req.time,req.price,req.quantity,True)
    # add to the ask dictionary
    ask_dictionary[req.orderid]=newask
    # put it in the queue
    ask_queue.put(newask)
    # update the customer's active ask list
    customer_dictionary[req.customerid].active_askids.append(req.orderid)
#
# deactivate a bid
#
def deactivate_bid(req):
    global bid_queue, customer_dictionary
    bidid=req.orderid
    customerid=req.customerid
    #
    # remove from the customer's bid list
    #
    oldlist=customer_dictionary[customerid].active_bidids
    newlist=[b for b in oldlist if b!=bidid]
    customer_dictionary[customerid].active_bidids=newlist
    #
    # make bid inactive
    #
    bid_dictionary[bidid].active=False
    
#
# deactivate an ask
#
def deactivate_ask(req):
    global ask_queue, customer_dictionary
    askid=req.orderid
    customerid=req.customerid
    #
    # remove from the customer's bid list
    #
    oldlist=customer_dictionary[customerid].active_askids
    newlist=[a for a in oldlist if a!=askid]
    customer_dictionary[customerid].active_askids=newlist
    #
    # make bid inactive
    #
    ask_dictionary[askid].active=False
    
#
# process a request based on a dataline
#
def process_request(req):
    global customer_dictionary
    #
    # check if this is a new customer and if it is 
    # add to dictionary with 0 cash and 0 nshares
    #
    cid=req.customerid
    if cid not in customer_dictionary.keys():
        cu=customer(cid,0,0,[],[])
        customer_dictionary[cid]=cu
    #
    # process the request
    #
    if req.activate_or_deactivate=="activate":
        if req.bid_or_ask=="bid":
            activate_new_bid(req)
        else:
            activate_new_ask(req)
    else:
        if req.bid_or_ask=="bid":
            deactivate_bid(req)
        else:
            deactivate_ask(req)


#
# process string of request line and return a request object
#
def create_request_from_string(linestr):
    linestr=linestr.replace("\n","")
    L=linestr.split(",")
    hr=int(L[0][0:2])
    minute=int(L[0][3:5])
    sec=int(L[0][6:8])
    ms=int(L[0][9:12])
    time=timeofday(hr,minute,sec,ms)
    customerid=L[1]
    activate_or_deactivate=L[2]
    bid_or_ask=L[3]
    orderid=L[4]
    if L[5]!="None":
        price=int(L[5])
    else:
        price=None
    if L[6]!="None":
        quantity=int(L[6])
    else:
        quantity=None
    req=request(time,customerid,activate_or_deactivate,bid_or_ask,orderid,price,quantity)
    return(req)

#
# get the current bid queue as a string
#
def bid_queue_string():
    global bid_queue
    bid_queue_str=""
    qtemp=Q.PriorityQueue()
    for i in range(len(bid_queue.queue)):
        tbid = bid_queue.get()
        if tbid.active:
            bid_queue_str+=tbid.__str__()
        qtemp.put(tbid)
    bid_queue=qtemp
    return(bid_queue_str)
#
# get the current ask queue as a string
#
def ask_queue_string():
    global ask_queue
    ask_queue_str=""
    qtemp=Q.PriorityQueue()
    for i in range(len(ask_queue.queue)):
        task = ask_queue.get()
        if task.active:
            ask_queue_str+=task.__str__()
        qtemp.put(task)
    ask_queue=qtemp
    return(ask_queue_str)

#
# MAIN PROGRAM
#
# get the requests from the requests file 
# and process them one at a time
#
with open("requests.csv","r") as fin:
    for line in fin:
        req=create_request_from_string(line)
        process_request(req)
        update_queue()             
#
# print out customers and their holdings
#
with open("customer_balances.csv","w") as fout:
    for cid in sorted(customer_dictionary.keys()):
        cu=customer_dictionary[cid]
        fout.write(cid+","+str(cu.cash)+","+str(cu.nshares)+"\n")

print("number of trades processed = "+str(trades_processed))
    


number of trades processed = 25539
